In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

import pdblp
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay

if (path := 'C:\\dev\\projects\\credit\\backend') not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
from api.data.base import tables, data_path
from api.data.fields import fields_history

In [ ]:
root_path = Path('/dev/projects/credit/backend/')
data_path = root_path / 'data'

### Bloomberg Connection

In [37]:
fields_history = [
    {
        "field": "PX_LAST",
        "name": "last_price",
        "overrides": [
            ('PRICING_SOURCE', 'CBBT')
        ]
    },
    {
        "field": "YLD_YTC_MID",
        "name": "yield_to_call",
        "overrides": [
            ('PRICING_SOURCE', 'CBBT')
        ]
    },
    {
        "field": "YLD_YTM_MID",
        "name": "ytm_flat",
        "overrides": [
            ('PRICING_SOURCE', 'CBBT')
        ]
    },
    {
        "field": "Z_SPRD_MID",
        "name": "z_spread",
        "overrides": [
            ('PRICING_SOURCE', 'CBBT')
        ]
    },
        {
        "field": "ASSET_SWAP_SPD_MID",
        "name": "asw_spread",
        "overrides": [
            ('PRICING_SOURCE', 'CBBT')
        ]
    },
]

In [61]:
dt_fmt = '%Y%m%d'
bq = BlpQuery(timeout=20000).start()

bonds_ref = tables.bonds_reference.reset_index()

securities = bonds_ref['security'].drop_duplicates().to_list()
fields = [field['field'] for field in fields_history]

start_date = pd.Timestamp(2007, 1, 1)
end_date = pd.Timestamp.today()

In [55]:
def add_pcs(securities, source):
    if isinstance(securities, str):
        securities = [securities]
    new_sec = []
    for sec in securities:
        root, yellow_key = sec.rsplit(' ', 1)
        new_sec.append(f"{root}@{source} {yellow_key}")
    
    return new_sec if len(new_sec) > 1 else new_sec[0]

res_cbbt = bq.bdh(
    securities=add_pcs(securities, 'CBBT'),
    fields=fields,
    overrides=[('PRICING_SOURCE', 'CBBT')],
    start_date=start_date.strftime(dt_fmt), 
    end_date=end_date.strftime(dt_fmt)
)

In [42]:
res.query('security== "ZR767554 Corp"')

date       security  last_price yield_to_call ytm_flat z_spread  \
713921  2019-09-27  ZR767554 Corp     101.538         3.905     4.98  431.543   
715750  2019-09-30  ZR767554 Corp     101.732         3.862    4.956  427.413   
716562  2019-10-01  ZR767554 Corp     102.045         3.793    4.917  420.464   
717105  2019-10-02  ZR767554 Corp     102.036         3.794    4.919  420.449   
718603  2019-10-03  ZR767554 Corp     101.977         3.807    4.927  423.779   
...            ...            ...         ...           ...      ...      ...   
1822118 2022-06-01  ZR767554 Corp      98.922          4.74    5.776   351.48   
1824239 2022-06-02  ZR767554 Corp      98.858         4.771    5.789  348.756   
1827588 2022-06-03  ZR767554 Corp      98.830         4.785    5.794  344.311   
1828066 2022-06-06  ZR767554 Corp      98.980         4.716    5.769  336.193   
1831287 2022-06-07  ZR767554 Corp      98.949         4.731    5.775      NaN   

        asw_spread  
713921     429.374  
715750     425.568  
716562     419.541  
717105     419.516  
718603     422.774  
...            ...  
1822118     343.47  
1824239    340.786  
1827588    336.434  
1828066    328.045  
1831287        NaN  

[691 rows x 7 columns]

In [56]:
res_cbbt = res_cbbt.rename({field['field']: field['name'] for field in fields_history}, axis=1)

In [70]:
res_cbbt['security'] = res_cbbt['security'].str.replace('@CBBT', '')
res_cbbt['pricing_source'] = 'CBBT'

In [69]:
no_cbbt = bonds_ref.loc[~bonds_ref['security'].isin(res_cbbt['security'].str.replace('@CBBT', '').unique())]

In [73]:
res_cbbt.to_clipboard()

In [ ]:
new_indices = indices_ref.loc[~indices_ref['ticker'].isin(indices_hist['security'].unique())]
# new_indices = new_indices.loc[new_indices['fields'] != "['CONTRBTD_ZSPREAD']", :]

In [ ]:
new_indices

Empty DataFrame
Columns: [ticker, name, fields, scale]
Index: []

In [ ]:
if not new_indices.empty:
    ress = {}

    for index, row in new_indices.iterrows():
        print(f"downloading data for {row['ticker']}")
        res = bq.bdh(
            securities=[row['ticker']],
            fields=eval(row['fields']),
            start_date=start_date.strftime(dt_fmt), 
            end_date=end_date.strftime(dt_fmt)
        )
        res = res.rename(name_mapping, axis=1)
        res['z_spread'] = res['z_spread'] / row['scale']
        ress[row['ticker']] = res 
    new_hist = pd.concat(ress.values(), keys=None)
    indices_hist = pd.concat([indices_hist, new_hist])\
                     .sort_values(['security', 'date'])\
                     .reset_index(drop=True)
    indices_hist.to_csv(data_path / 'indices_history.csv', index=False)

downloading data for I03139EU Index
downloading data for SUBFIN CDSI GEN 5Y Corp
downloading data for SNRFIN CDSI GEN 5Y Corp
downloading data for CDX IG CDSI GEN 5Y Corp
downloading data for CDX HY CDSI GEN 5Y SPRD Corp
downloading data for BLQCTRUU Index
downloading data for BHYOTRUU Index


In [74]:
from api.gui.utils import make_data_table

In [75]:
dt = make_data_table(bonds_ref)

In [77]:
t = dt.table

In [78]:

def test(widget, event, payload):
    global p 
    p = payload

In [84]:
p

{'index': 0,
 'security': 'ZR767554 Corp',
 'ticker': 'BAMIIM',
 'name': 'BAMIIM 4 1/4 10/01/29',
 'isin': 'XS2058908745',
 'currency': 'EUR',
 'country': 'IT',
 'bics_sector': 'Financials',
 'bics_industry': 'Banks',
 'amount_outstanding': '350000000.00',
 'issue_date': '2019-10-01',
 'maturity': '2029-10-01 00',
 'calc_maturity': '2029-10-01',
 'workout_date': '2024-10-01',
 'maturity_type': 'CALLABLE',
 'coupon': '4.25',
 'coupon_type': 'VARIABLE',
 'rating_sp': '',
 'rating_moody': 'Ba3',
 'rating_fitch': '',
 'rating_bbg': 'BB-',
 'is_fixed': 'N',
 'is_callable': 'Y',
 'is_perpetual': 'N',
 'is_convertible': 'N',
 'is_subordinated': 'Y',
 'payment_rank': 'Subordinated',
 'bail_in_bond_designation': 'Insufficient Information',
 'basel_iii_designation': 'Tier 2',
 'is_green': 'N',
 'custom_rating': 'BB-',
 'capital_stack': 'lt2',
 'rating_bucket': 'BB',
 'macroarea': 'Europe',
 'area': 'Western Europe',
 'reset_index': 'EUSA5',
 'issuer_equity_ticker': 'BAMI IM Equity',
 'cds_ticker

In [82]:
t.on_event('click:row', test)

In [83]:
t

DataTable(class_='elevation-1', dense=True, headers=[{'text': 'index', 'value': 'index', 'align': 'start', 'wi…

In [ ]:
# indices_hist.loc[indices_hist['security'] == 'BUS3TRUU Index']

date        security  z_spread
11640 2007-01-02  BUS3TRUU Index  0.665987
11641 2007-01-03  BUS3TRUU Index  0.665461
11642 2007-01-04  BUS3TRUU Index  0.664393
11643 2007-01-05  BUS3TRUU Index  0.658689
11644 2007-01-08  BUS3TRUU Index  0.658635
...          ...             ...       ...
15494 2022-05-18  BUS3TRUU Index  1.182188
15495 2022-05-19  BUS3TRUU Index  1.183101
15496 2022-05-20  BUS3TRUU Index  1.200134
15497 2022-05-23  BUS3TRUU Index  1.186637
15498 2022-05-24  BUS3TRUU Index  1.173695

[3859 rows x 3 columns]